**Scrapping and Wrangling the data of Toronto Neighborhoods**

In [1]:
#!pip install folium

In [2]:
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab

In [3]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import geocoder

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [4]:
url= "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
data= requests.get(url).text
data= BeautifulSoup(data, "lxml")
table=data.find('table')
table_contents= []
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


**Reading Geospatial Coordinates CSV file with Watson Studio**

In [5]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


**Merging the data to add Latitude and Longitude to the Data Frame**

In [6]:
geodata.rename(columns={"Postal Code": "PostalCode"}, inplace= True)
toronto_data = pd.merge(df, geodata, how="left", on="PostalCode")
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [7]:
north_york_data= toronto_data[toronto_data["Borough"]=="North York"].reset_index(drop= True)
north_york_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
3,M3B,North York,Don Mills North,43.745906,-79.352188
4,M6B,North York,Glencairn,43.709577,-79.445073


**Visualizing Map of North York**

In [8]:
address = 'North York'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of North York are 43.7543263, -79.44911696639593.


In [9]:
# create map of North York using latitude and longitude values
map_northyork = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(north_york_data['Latitude'], north_york_data['Longitude'], north_york_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_northyork)  
    
map_northyork

**Using FourSquare API to Explore Neighborhoods in North York**

In [10]:
# @hidden
CLIENT_ID = '4IPUG2WMXR3SIJA1WQTFXECDJCXP2EFLXOYYJHEHR5E1FDPG' # your Foursquare ID
CLIENT_SECRET = 'RC2V1OEHTYCPW3FDDC31CGVPI2HRZJQJA0CTY4JAKCG45SMT' # your Foursquare Secret
ACCESS_TOKEN = 'JD4E4UUAD4TCHGEAQCQWJYVAQAWRJF4KP2KICYZ1GCBTHJG3' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 4IPUG2WMXR3SIJA1WQTFXECDJCXP2EFLXOYYJHEHR5E1FDPG
CLIENT_SECRET:RC2V1OEHTYCPW3FDDC31CGVPI2HRZJQJA0CTY4JAKCG45SMT


In [11]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [12]:
north_york_venues = getNearbyVenues(names=north_york_data['Neighborhood'],
                                   latitudes=north_york_data['Latitude'],
                                   longitudes=north_york_data['Longitude']
                                  )

In [13]:
print(north_york_venues.shape)
north_york_venues.head()

(203, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
1,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [14]:
print('There are {} unique categories.'.format(len(north_york_venues['Venue Category'].unique())))
north_york_venues.groupby('Neighborhood').count()

There are 88 unique categories.


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Wilson Heights, Downsview North",22,22,22,22,22,22
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",26,26,26,26,26,26
Don Mills North,4,4,4,4,4,4
Don Mills South,20,20,20,20,20,20
Downsview Central,3,3,3,3,3,3
Downsview East,2,2,2,2,2,2
Downsview Northwest,4,4,4,4,4,4
Downsview West,5,5,5,5,5,5


**One-Hot Encoding**

In [15]:
# one hot encoding
north_york_onehot = pd.get_dummies(north_york_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
north_york_onehot['Neighborhood'] = north_york_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [north_york_onehot.columns[-1]] + list(north_york_onehot.columns[:-1])
north_york_onehot = north_york_onehot[fixed_columns]

north_york_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Boutique,Bridal Shop,Bubble Tea Shop,Burrito Place,Bus Line,Business Service,Butcher,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Event Space,Falafel Restaurant,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hobby Shop,Hockey Arena,Home Service,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Liquor Store,Massage Studio,Mediterranean Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shopping Mall,Sporting Goods Shop,Supermarket,Sushi Restaurant,Thai Restaurant,Toy / Game Store,Trail,Video Game Store,Vietnamese Restaurant
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


**Grouping Neighborhoods**

In [16]:
north_york_grouped = north_york_onehot.groupby('Neighborhood').mean().reset_index()
north_york_grouped.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Boutique,Bridal Shop,Bubble Tea Shop,Burrito Place,Bus Line,Business Service,Butcher,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Event Space,Falafel Restaurant,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hobby Shop,Hockey Arena,Home Service,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Liquor Store,Massage Studio,Mediterranean Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shopping Mall,Sporting Goods Shop,Supermarket,Sushi Restaurant,Thai Restaurant,Toy / Game Store,Trail,Video Game Store,Vietnamese Restaurant
0,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.000000,0.00,0.0,0.00,0.0,0.0,0.090909,0.0,0.0,0.0,0.00,0.00,0.0,0.045455,0.0,0.0,0.00,0.0,0.000000,0.000000,0.00,0.00,0.00,0.090909,0.000000,0.0,0.0,0.000000,0.045455,0.00,0.045455,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.045455,0.045455,0.0,0.045455,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.0,0.045455,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.045455,0.0,0.045455,0.0,0.045455,0.0,0.045455,0.045455,0.0,0.0,0.0,0.000000,0.0,0.045455,0.0,0.045455,0.045455,0.00,0.045455,0.045455,0.000000,0.000000,0.0,0.0,0.0
1,Bayview Village,0.0,0.0,0.000000,0.00,0.0,0.00,0.0,0.0,0.250000,0.0,0.0,0.0,0.00,0.00,0.0,0.000000,0.0,0.0,0.00,0.0,0.000000,0.250000,0.00,0.25,0.00,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.00,0.000000,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
2,"Bedford Park, Lawrence Manor East",0.0,0.0,0.038462,0.00,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.00,0.0,0.000000,0.0,0.0,0.00,0.0,0.038462,0.038462,0.00,0.00,0.00,0.076923,0.038462,0.0,0.0,0.038462,0.000000,0.00,0.000000,0.00,0.0,0.0,0.0,0.0,0.038462,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.038462,0.038462,0.00,0.0,0.038462,0.0,0.0,0.000000,0.038462,0.000000,0.076923,0.038462,0.038462,0.038462,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.038462,0.038462,0.0,0.0,0.0,0.038462,0.0,0.038462,0.0,0.076923,0.000000,0.00,0.000000,0.038462,0.038462,0.038462,0.0,0.0,0.0
3,Don Mills North,0.0,0.0,0.000000,0.00,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.00,0.0,0.000000,0.0,0.0,0.00,0.0,0.000000,0.250000,0.25,0.00,0.00,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.00,0.000000,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.25,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
4,Don Mills South,0.0,0.0,0.000000,0.05,0.0,0.05,0.0,0.0,0.000000,0.0,0.0,0.0,0.05,0.05,0.0,0.000000,0.0,0.0,0.05,0.0,0.000000,0.000000,0.00,0.05,0.05,0.100000,0.000000,0.0,0.0,0.000000,0.000000,0.05,0.000000,0.05,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.050000,0.10,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.050000,0.000000,0.000000,0.00

**Getting the Top 7 Most Common Venues in Each Neighborhood**

In [17]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [36]:
num_top_venues = 7

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = north_york_grouped['Neighborhood']

for ind in np.arange(north_york_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(north_york_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
0,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Frozen Yogurt Shop,Deli / Bodega,Ice Cream Shop,Bridal Shop,Middle Eastern Restaurant
1,Bayview Village,Café,Bank,Japanese Restaurant,Chinese Restaurant,Fast Food Restaurant,Falafel Restaurant,Clothing Store
2,"Bedford Park, Lawrence Manor East",Italian Restaurant,Sandwich Place,Coffee Shop,Comfort Food Restaurant,Pizza Place,Butcher,Japanese Restaurant
3,Don Mills North,Café,Japanese Restaurant,Gym,Caribbean Restaurant,Dim Sum Restaurant,Clothing Store,Coffee Shop
4,Don Mills South,Restaurant,Coffee Shop,Gym,Bus Line,Italian Restaurant,Bike Shop,Beer Store


**Clustering Neighborhoods with K Means**

In [37]:
north_york_grouped_clustering = north_york_grouped.drop('Neighborhood', 1)
# set number of clusters
kclusters = 5



# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(north_york_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 0, 1, 0, 1, 3, 2, 1, 0, 1], dtype=int32)

**Merging the Cluster label to the Data Frame**

In [38]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

north_york_merged = north_york_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
north_york_merged = north_york_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

north_york_merged= north_york_merged.dropna(0).reset_index()

north_york_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1.0,Park,Food & Drink Shop,Fast Food Restaurant,Vietnamese Restaurant,Deli / Bodega,Chinese Restaurant,Clothing Store
1,M4A,North York,Victoria Village,43.725882,-79.315572,1.0,Coffee Shop,Pizza Place,Intersection,Portuguese Restaurant,Hockey Arena,Vietnamese Restaurant,Cosmetics Shop
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1.0,Clothing Store,Furniture / Home Store,Vietnamese Restaurant,Miscellaneous Shop,Boutique,Coffee Shop,Event Space
3,M3B,North York,Don Mills North,43.745906,-79.352188,0.0,Café,Japanese Restaurant,Gym,Caribbean Restaurant,Dim Sum Restaurant,Clothing Store,Coffee Shop
4,M6B,North York,Glencairn,43.709577,-79.445073,0.0,Park,Japanese Restaurant,Italian Restaurant,Bakery,Vietnamese Restaurant,Deli / Bodega,Clothing Store


### Examing the Individual Clusters

**First Cluster**

In [ ]:
north_york_merged.loc[north_york_merged['Cluster Labels'] == 0, north_york_merged.columns[[2] + list(range(6, north_york_merged.shape[1]))]]

**Second Cluster**

In [ ]:
north_york_merged.loc[north_york_merged['Cluster Labels'] == 1, north_york_merged.columns[[2] + list(range(6, north_york_merged.shape[1]))]]

**Third Cluster**

In [ ]:
north_york_merged.loc[north_york_merged['Cluster Labels'] == 2, north_york_merged.columns[[2] + list(range(6, north_york_merged.shape[1]))]]

**Fourth Cluster**

In [ ]:
north_york_merged.loc[north_york_merged['Cluster Labels'] == 3, north_york_merged.columns[[2] + list(range(6, north_york_merged.shape[1]))]]

**Fifth Cluster**

In [ ]:
north_york_merged.loc[north_york_merged['Cluster Labels'] == 4, north_york_merged.columns[[2] + list(range(6, north_york_merged.shape[1]))]]

**Visualizing the Clusters**

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(north_york_merged['Latitude'], north_york_merged['Longitude'], north_york_merged['Neighborhood'], north_york_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters